<a href="https://colab.research.google.com/github/jvitorjoliveira/-imigracao/blob/main/DadosSISMIGRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .appName("SISMIGRA") \
    .getOrCreate()

In [5]:
# Carregar os arquivos no Colab
file_2018 = "/content/drive/MyDrive/SISMIGRA/sismigra_2018.csv"
file_2023 = "/content/drive/MyDrive/SISMIGRA/sismigra_2023.csv"

# Carregar os arquivos no PySpark
df_2018 = spark.read.csv(file_2018, header=True, inferSchema=True, encoding='latin1', sep=';')
df_2023 = spark.read.csv(file_2023, header=True, inferSchema=True, encoding='latin1', sep=';')

# Mostrar a estrutura dos dados
df_2018.printSchema()
df_2023.printSchema()

root
 |-- DATAREGISTRO: integer (nullable = true)
 |-- AMPARO: string (nullable = true)
 |-- AMPARO_DESCRICAO: string (nullable = true)
 |-- GRUPO_AMPARO_LEGAL: string (nullable = true)
 |-- CLASSIFICACAO_REGISTRO: string (nullable = true)
 |-- MEIO_TRANSPORTE: string (nullable = true)
 |-- DATAENTRADA: integer (nullable = true)
 |-- UNIDADE_FED_ENTRADA: string (nullable = true)
 |-- UNIDADE_FED_RESIDENCIA: string (nullable = true)
 |-- MUNICIPIO_RESIDENCIA: string (nullable = true)
 |-- PAIS_NASCIMENTO: string (nullable = true)
 |-- PAIS_NACIONALIDADE: string (nullable = true)
 |-- CONTINENTE: string (nullable = true)
 |-- FAIXA_ETARIA: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- ESTADO_CIVIL: string (nullable = true)
 |-- PROFISSAO: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- CLASSIFICACAO_REVISADA: string (nullable = true)

root
 |-- DATAREGISTRO: integer (nullable = true)
 |-- AMPARO: integer (nullable = true)
 |-- AMPARO_DESCRICAO: str

In [6]:
# 1. QUANTIDADE NUMÉRICA DE IMIGRANTES POR PAÍS DE ORIGEM (2018, 2023, VAR. P.P.)

# Agrupando por nacionalidade e contando os registros
nacionalidade_18 = df_2018.groupBy('PAIS_NACIONALIDADE').count().withColumnRenamed("count", "total_2018")
nacionalidade_23 = df_2023.groupBy('PAIS_NACIONALIDADE').count().withColumnRenamed("count", "total_2023")

# Juntando os dois dfs por nacionalidade
from pyspark.sql.functions import coalesce, col

merged = nacionalidade_18.join(
    nacionalidade_23,
    on="PAIS_NACIONALIDADE",
    how="outer"
).fillna(0)

# Calculando a variação percentual
merged = merged.withColumn(
    "variacao_percentual",
    ((col("total_2023") - col("total_2018")) / col("total_2018") * 100).cast("double")
)

# Mostrando os resultados
merged.show(10)

+------------------+----------+----------+-------------------+
|PAIS_NACIONALIDADE|total_2018|total_2023|variacao_percentual|
+------------------+----------+----------+-------------------+
|                 -|         0|       238|               NULL|
|      AFEGANISTÃO|        30|      3231|            10670.0|
|     AFRICA DO SUL|        92|       204| 121.73913043478262|
|          ALBÃNIA|         7|         9|  28.57142857142857|
|          ALEMANHA|      1426|      1313| -7.924263674614306|
|           ANDORRA|         0|         3|               NULL|
|            ANGOLA|      1100|      3271| 197.36363636363637|
|          ANTÃGUA|         0|         7|               NULL|
|         APÃTRIDA|         4|         1|              -75.0|
|         ARGENTINA|      4894|      8931|  82.48876174908051|
+------------------+----------+----------+-------------------+
only showing top 10 rows



In [7]:
from pyspark.sql.functions import when, col, format_number

# "-" --> "Sem classificação"
merged = merged.withColumn(
    "PAIS_NACIONALIDADE",
    when(col("PAIS_NACIONALIDADE") == "-", "Sem classificação").otherwise(col("PAIS_NACIONALIDADE"))
)

# NULL --> "0"
merged = merged.withColumn(
    "variacao_percentual",
    when(col("variacao_percentual").isNull(), 0).otherwise(col("variacao_percentual"))
)

# Tranformando em objeto númerico
merged = merged.withColumn(
    "variacao_percentual",
    when(col("variacao_percentual") == "0", 0).otherwise(col("variacao_percentual").cast("double"))
)

# Formatando para uma casa decimal
merged = merged.withColumn(
    "variacao_percentual",
    format_number(col("variacao_percentual"), 1)
)

# Ordenando de forma decrescente pelo total de estrangeiros em 2023
merged_ordenado = merged.orderBy(col("total_2023").desc())

# Mostrando os resultados ordenados
1. análisemerged_ordenado.show(10)

+------------------+----------+----------+-------------------+
|PAIS_NACIONALIDADE|total_2018|total_2023|variacao_percentual|
+------------------+----------+----------+-------------------+
|         VENEZUELA|     32494|    126185|              288.3|
|          BOLÃVIA|      7862|     16175|              105.7|
|         COLÃMBIA|      9699|      9678|               -0.2|
|         ARGENTINA|      4894|      8931|               82.5|
|              CUBA|      1863|      7673|              311.9|
|          PARAGUAI|      2856|      7369|              158.0|
|             HAITI|     14227|      6495|              -54.3|
|              PERU|      3055|      4168|               36.4|
|           URUGUAI|      5597|      3486|              -37.7|
|    ESTADOS UNIDOS|      3253|      3286|                1.0|
+------------------+----------+----------+-------------------+
only showing top 10 rows



In [8]:
# 2. UF DE RESIDÊNCIA DOS IMIGRANTES (TOTAL NUMÉRICO) -- 2018, 2023 E VAR P.P.

from pyspark.sql.functions import col, when, format_number

# Agrupando por UF e contando os registros (2018 vs. 2023)
uf_18 = df_2018.groupBy('UNIDADE_FED_RESIDENCIA').count().withColumnRenamed("count", "total_2018")
uf_23 = df_2023.groupBy('UNIDADE_FED_RESIDENCIA').count().withColumnRenamed("count", "total_2023")

# Juntar os dois DataFrames por UF
uf_merged = uf_18.join(uf_23, on="UNIDADE_FED_RESIDENCIA", how="outer").fillna(0)

# Calculando a variação percentual
uf_merged = uf_merged.withColumn(
    "variacao_percentual",
    when(col("total_2018") == 0, 0).otherwise(((col("total_2023") - col("total_2018")) / col("total_2018") * 100).cast("double"))
)

# Formatando a variação percentual para uma casa decimal
uf_merged = uf_merged.withColumn(
    "variacao_percentual",
    format_number(col("variacao_percentual"), 1)
)

# Ordenando de forma decrescente pelo total de imigrantes em 2023
uf_sorted = uf_merged.orderBy(col("total_2023").desc())

# Mostrando os resultados
uf_sorted.show(10)

+----------------------+----------+----------+-------------------+
|UNIDADE_FED_RESIDENCIA|total_2018|total_2023|variacao_percentual|
+----------------------+----------+----------+-------------------+
|                    SP|     34877|     50814|               45.7|
|                    RR|     23564|     45197|               91.8|
|                    SC|      7972|     28650|              259.4|
|                    PR|      8301|     25685|              209.4|
|                    AM|      4234|     17900|              322.8|
|                    RS|     10224|     17510|               71.3|
|                    RJ|      8800|      9399|                6.8|
|                    MG|      4535|      6325|               39.5|
|                    MS|      1476|      5171|              250.3|
|                    MT|      1309|      4432|              238.6|
+----------------------+----------+----------+-------------------+
only showing top 10 rows



In [9]:
# 3. QUANTIDADE DE IMIGRANTES POR REGIÃO BRASILEIRA (2018, 2023, VAR P.P.)

from pyspark.sql.functions import col, when, format_number

# De estados --> regiões brasileiras
regioes = {
    "Norte": ["AC", "AP", "AM", "PA", "RO", "RR", "TO"],
    "Nordeste": ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
    "Centro-Oeste": ["DF", "GO", "MT", "MS"],
    "Sudeste": ["ES", "MG", "RJ", "SP"],
    "Sul": ["PR", "RS", "SC"]
}

# Função para mapear estados às suas regiões
def mapear_regiao(uf_col):
    return when(uf_col.isin(*regioes["Norte"]), "Norte") \
           .when(uf_col.isin(*regioes["Nordeste"]), "Nordeste") \
           .when(uf_col.isin(*regioes["Centro-Oeste"]), "Centro-Oeste") \
           .when(uf_col.isin(*regioes["Sudeste"]), "Sudeste") \
           .when(uf_col.isin(*regioes["Sul"]), "Sul") \
           .otherwise("Outros")  # Caso haja algum valor não identificado

# Adicionar a coluna de região em 2018 e 2023
df_2018 = df_2018.withColumn("REGIAO", mapear_regiao(col("UNIDADE_FED_RESIDENCIA")))
df_2023 = df_2023.withColumn("REGIAO", mapear_regiao(col("UNIDADE_FED_RESIDENCIA")))

# Agrupar por região e contar os registros
regiao_18 = df_2018.groupBy("REGIAO").count().withColumnRenamed("count", "total_2018")
regiao_23 = df_2023.groupBy("REGIAO").count().withColumnRenamed("count", "total_2023")

# Juntar os dois dfs por região
regiao_merged = regiao_18.join(regiao_23, on="REGIAO", how="outer").fillna(0)

# Calcular a variação percentual
regiao_merged = regiao_merged.withColumn(
    "variacao_percentual",
    when(col("total_2018") == 0, 0).otherwise(((col("total_2023") - col("total_2018")) / col("total_2018") * 100).cast("double"))
)

# Formatar a variação percentual para uma casa decimal
regiao_merged = regiao_merged.withColumn(
    "variacao_percentual",
    format_number(col("variacao_percentual"), 1)
)

# Ordenar de forma decrescente pelo total de imigrantes (2023)
regiao_sorted = regiao_merged.orderBy(col("total_2023").desc())

# Mostrar os resultados
regiao_sorted.show()

+------------+----------+----------+-------------------+
|      REGIAO|total_2018|total_2023|variacao_percentual|
+------------+----------+----------+-------------------+
|         Sul|     26497|     71845|              171.1|
|       Norte|     29888|     68140|              128.0|
|     Sudeste|     48926|     67545|               38.1|
|Centro-Oeste|      6026|     14924|              147.7|
|    Nordeste|      9103|      8994|               -1.2|
|      Outros|      1334|       745|              -44.2|
+------------+----------+----------+-------------------+



In [10]:
# IMIGRANTES POR UF

df_2023.select("UNIDADE_FED_RESIDENCIA").distinct().show()
imigrantes_estado_df = df_2023.groupBy("UNIDADE_FED_RESIDENCIA").count().withColumnRenamed("count", "imigrantes_2023")
imigrantes_estado_df.show()

+----------------------+
|UNIDADE_FED_RESIDENCIA|
+----------------------+
|                    SC|
|                    RO|
|                    PI|
|                    AM|
|                    RR|
|                    GO|
|                    TO|
|                    MT|
|                    SP|
|                    ES|
|                    PB|
|                    RS|
|                    MS|
|                    AL|
|                     -|
|                    MG|
|                    PA|
|                    BA|
|                    SE|
|                    PE|
+----------------------+
only showing top 20 rows

+----------------------+---------------+
|UNIDADE_FED_RESIDENCIA|imigrantes_2023|
+----------------------+---------------+
|                    SC|          28650|
|                    RO|           2426|
|                    PI|            285|
|                    AM|          17900|
|                    RR|          45197|
|                    GO|           3047|
|    

In [11]:
# 4. PERCENTUAL DE IMIGRANTES within POPULAÇÃO ESTADUAL (DE ACORDO COM O CENSO DO IBGE 2022)

from pyspark.sql.functions import format_number

# Dicionário com a população de cada estado (Censo 2022)
censo_2022 = {
    "SP": 44411238, "MG": 20538718, "RJ": 16054524, "BA": 14141626, "PR": 11444380,
    "RS": 10882965, "PE": 9058931, "CE": 8794957, "PA": 8121025, "SC": 7610361,
    "GO": 7056495, "MA": 6775805, "AM": 3941613, "PB": 3974687, "ES": 3833712,
    "MT": 3658649, "RN": 3302729, "PI": 3271199, "AL": 3127683, "DF": 2817381,
    "MS": 2757013, "SE": 2209558, "RO": 1581196, "TO": 1511460, "AC": 830018,
    "AP": 733759, "RR": 636707
}

# Criando um DataFrame com a população de cada estado para o Censo (2022)
from pyspark.sql import Row
pop_estado_df = spark.createDataFrame([Row(UNIDADE_FED_RESIDENCIA=k, pop_censo_2022=v)
                                      for k, v in censo_2022.items()])

# Somando a quantidade de imigrantes por estado no SISMIGRA (2023)
imigrantes_estado_df = df_2023.groupBy("UNIDADE_FED_RESIDENCIA").count().withColumnRenamed("count", "imigrantes_2023")

# Juntando os dois dfs
df_estado_comparacao = imigrantes_estado_df.join(pop_estado_df, on="UNIDADE_FED_RESIDENCIA", how="inner")

# Calculando a % de imigrantes em relação à população do Censo (2022)
df_estado_comparacao = df_estado_comparacao.withColumn(
    "percentual_imigrantes",
    (col("imigrantes_2023") / col("pop_censo_2022") * 100)
)

# Ordenando pelos estados com maior percentual de imigrantes
df_estado_comparacao_sorted = df_estado_comparacao.orderBy(col("percentual_imigrantes").desc())

# Formatando a coluna c/ 2 casas decimais
df_estado_comparacao_sorted = df_estado_comparacao_sorted.withColumn(
    "percentual_imigrantes",
    format_number(col("percentual_imigrantes"), 2)
)

# Mostrando o resultado
df_estado_comparacao_sorted.select("UNIDADE_FED_RESIDENCIA", "imigrantes_2023", "pop_censo_2022", "percentual_imigrantes").show(10)

+----------------------+---------------+--------------+---------------------+
|UNIDADE_FED_RESIDENCIA|imigrantes_2023|pop_censo_2022|percentual_imigrantes|
+----------------------+---------------+--------------+---------------------+
|                    RR|          45197|        636707|                 7.10|
|                    AM|          17900|       3941613|                 0.45|
|                    SC|          28650|       7610361|                 0.38|
|                    PR|          25685|      11444380|                 0.22|
|                    MS|           5171|       2757013|                 0.19|
|                    RS|          17510|      10882965|                 0.16|
|                    RO|           2426|       1581196|                 0.15|
|                    MT|           4432|       3658649|                 0.12|
|                    SP|          50814|      44411238|                 0.11|
|                    AC|            771|        830018|         

In [12]:
# 5. AGRUPAMENTO POR FAIXA ETÁRIA (2018, 2023 e MUDANÇA %)

import pandas as pd

# Caminhos dos arquivos
file_2018 = "/content/drive/MyDrive/SISMIGRA/sismigra_2018.csv"
file_2023 = "/content/drive/MyDrive/SISMIGRA/sismigra_2023.csv"

# Carregando os dados
dados_2018 = pd.read_csv(file_2018, delimiter=";")
dados_2023 = pd.read_csv(file_2023, delimiter=";")

# Agrupando por faixa etária e contar registros
faixa_2018 = dados_2018.groupby('FAIXA_ETARIA').size()
faixa_2023 = dados_2023.groupby('FAIXA_ETARIA').size()

# Criando df para comparações
comparacao = pd.DataFrame({
    '2018': faixa_2018,
    '2023': faixa_2023
}).fillna(0)  # Substitui ausentes por 0

# Calculando crescimento percentual
comparacao['Crescimento %'] = ((comparacao['2023'] - comparacao['2018']) / comparacao['2018']) * 100

# Ordenando os resultados com uma casa decimal
comparacao = comparacao.sort_values('Crescimento %', ascending=False)
comparacao['Crescimento %'] = comparacao['Crescimento %'].round(1)

# Exibindo
print(comparacao)

<ipython-input-12-fd2f4fb26817>:8: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_2018 = pd.read_csv(file_2018, delimiter=";")


                   2018   2023  Crescimento %
FAIXA_ETARIA                                 
NÃO ESPECIFICADO    102  16734        16305.9
0 |-- 15 anos     11195  46199          312.7
65 |-- anos        2005   6929          245.6
40 |-- 65 anos    23835  41059           72.3
15 |-- 25 anos    33761  54896           62.6
25 |-- 40 anos    50876  66376           30.5


In [13]:
# 6. AGRUPAMENTO POR PROFISSÃO (2018, 2023 e MUDANÇA %)

import pandas as pd

# Carregando os dados
dados_2018 = pd.read_csv("/content/drive/MyDrive/SISMIGRA/sismigra_2018.csv", delimiter=";")
dados_2023 = pd.read_csv("/content/drive/MyDrive/SISMIGRA/sismigra_2023.csv", delimiter=";")

# Agrupando por profissão e contando registros
profissao_2018 = dados_2018.groupby('PROFISSAO').size()
profissao_2023 = dados_2023.groupby('PROFISSAO').size()

# Criando df para comparações
comparacao_profissao = pd.DataFrame({
    '2018': profissao_2018,
    '2023': profissao_2023
}).fillna(0)  # Substitui ausentes por 0

# Calculando o crescimento percentual
comparacao_profissao['Crescimento %'] = ((comparacao_profissao['2023'] - comparacao_profissao['2018']) / comparacao_profissao['2018']) * 100

# Ordenando os resultados por crescimento percentual, em ordem decrescente
comparacao_profissao = comparacao_profissao.sort_values('Crescimento %', ascending=False)

# Exibindo o crescimento percentual com uma casa decimal
comparacao_profissao['Crescimento %'] = comparacao_profissao['Crescimento %'].round(1)

# Exibindo o resultado
print(comparacao_profissao)

<ipython-input-13-9c48ad22ad44>:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_2018 = pd.read_csv("/content/drive/MyDrive/SISMIGRA/sismigra_2018.csv", delimiter=";")


                                                      2018     2023  \
PROFISSAO                                                             
-                                                      0.0     23.0   
MENOR ( CRIANCA , NAO ESTUDANTE )                   2690.0  24297.0   
SEM OCUPACAO                                        8874.0  35093.0   
PESCADOR, LENHADOR, OU ASSEMELHADO                    55.0    201.0   
MANEQUIM, MODELO, OU ASSEMELHADO                      12.0     42.0   
...                                                    ...      ...   
AERONAUTA, PILOTO, NAVEGADOR, COMISSARIO, AEROM...   167.0     78.0   
TRABALHADOR NA INDUSTRIA DO FUMO                      14.0      4.0   
TRABALHADOR DA INDUSTRIA QUIMICA, DA INDUSTRIA ...   200.0     57.0   
DIRETOR , GERENTE OU PROPRIETARIO, DE ESTABELEC...   543.0    148.0   
OCUPANTE DE CARGO LEGISLATIVO, ( SENADOR , DEPU...     1.0      0.0   

                                                    Crescimento %  
PROFISSA

In [14]:
# 7. PRINCIPAL PROFISSÃO POR PAÍS (2018, 2023 e MUDANÇA %)

import pandas as pd

# Carregando os dados
df_2018 = pd.read_csv("/content/drive/MyDrive/SISMIGRA/sismigra_2018.csv", delimiter=";")
df_2023 = pd.read_csv("/content/drive/MyDrive/SISMIGRA/sismigra_2023.csv", delimiter=";")

# Concatenando os dados de 2018 e 2023 para análise conjunta
df = pd.concat([df_2018, df_2023])

# Agrupando os dados por país de origem e profissão em 2023
agrup_prof_2023 = df_2023.groupby(['PAIS_NACIONALIDADE', 'PROFISSAO']).size().unstack(fill_value=0)

# Filtrando países com no mínimo 50 imigrantes (em 2023)
imigrantes_pais_2023 = df_2023['PAIS_NACIONALIDADE'].value_counts()
pais_50_ou_mais = imigrantes_pais_2023[imigrantes_pais_2023 >= 50].index

# Filtrando o df para incluir apenas os países com, no mínimo, 50 imigrantes
agrup_pais_prof_2023 = agrup_prof_2023.loc[pais_50_ou_mais]

# Identificando a profissão com maior número de imigrantes p/ cada país
prof_pais = agrup_pais_prof_2023.idxmax(axis=1)

# Calculando o número bruto de imigrantes para a profissão predominante
num_imigrantes_prof = agrup_pais_prof_2023.max(axis=1)

# Criando o df final
top_3_prof = pd.DataFrame({
    'Pais': num_imigrantes_prof.index,
    'Profissao': prof_pais,
    'Num Imigrantes': num_imigrantes_prof
})

# Ordenando
top_3_prof_sorted = top_3_prof.sort_values(by='Num Imigrantes', ascending=False).head(3)

# Exibindo os resultados
print(top_3_prof_sorted)

<ipython-input-14-b1633ad83fa8>:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2018 = pd.read_csv("/content/drive/MyDrive/SISMIGRA/sismigra_2018.csv", delimiter=";")


                         Pais  \
PAIS_NACIONALIDADE              
VENEZUELA           VENEZUELA   
BOLÍVIA               BOLÍVIA   
CUBA                     CUBA   

                                                            Profissao  \
PAIS_NACIONALIDADE                                                      
VENEZUELA                                                SEM OCUPACAO   
BOLÍVIA             DECORADOR, COSTUREIRO, ALFAIATE, MODISTA, PELE...   
CUBA                                  OUTRA OCUPACAO NAO CLASSIFICADA   

                    Num Imigrantes  
PAIS_NACIONALIDADE                  
VENEZUELA                    24930  
BOLÍVIA                       7128  
CUBA                          3626  


In [ ]:
### TRANSFORMANDO A ÚLTIMA ANÁLISE EM UM DF .CSV

top_3_prof_sorted.to_csv('/content/drive/MyDrive/SISMIGRA/top_3_profissoes.csv', index=False)

In [ ]:
### RENOMEANDO COLUNAS

import pandas as pd

# Carregando os dados
df_2018 = pd.read_csv("/content/drive/MyDrive/SISMIGRA/sismigra_2018.csv", delimiter=";")

# Renomeando
df_2018.rename(columns={
    'DATAREGISTRO': 'Data_Registro',
    'AMPARO': 'Tipo_Amparo',
    'GRUPO_AMPARO_LEGAL': 'Grupo_Amparo_Legal',
    'CLASSIFICACAO_REGISTRO': 'Classificacao_Registro',
    'MEIO_TRANSPORTE': 'Meio_Transporte'
}, inplace=True)

# Exibindo
print(df_2018.head())